In [4]:
using JuMP, Gurobi

# Define the model, using Gurobi as the solver
model = Model(Gurobi.Optimizer)

# Parameters (Example values, adjust accordingly)
# Adjusted Parameters
N = 10  # Number of vehicles
M = 5   # Number of charging stations
T = 24  # Time periods (e.g., hours in a day)
B = [100 for i in 1:N]  # Battery capacities
I = [20, 40, 60, 50, 70, 50, 40, 60, 80, 50]   # Adjusted initial charge levels
D = [10, 15, 20, 25, 30, 20, 25, 15, 10, 20]   # Adjusted distances for trips
E = 0.01  # Energy consumption rate
P = [20 for j in 1:M]  # Charging rates
S = [3, 4, 3, 2, 4]   # Adjusted station capacities
# Station capacities (increased)

# Decision variables
@variable(model, x[i=1:N, j=1:M, t=1:T], Bin)  # Whether vehicle i charges at station j at time t
@variable(model, c[i=1:N, t=1:T] >= 0)         # Charge amount for vehicle i at time t

# Objective: Minimize total charging time
@objective(model, Min, sum(x[i,j,t] for i=1:N, j=1:M, t=1:T))

# Constraints
# Sufficient charge for trips
@constraint(model, [i=1:N], sum(c[i,t] for t=1:T) >= E * D[i])

# Charging station capacity
@constraint(model, [j=1:M, t=1:T], sum(x[i,j,t] for i=1:N) <= S[j])

# Battery capacity limit
@constraint(model, [i=1:N, t=1:T], I[i] + sum(c[i,tp] for tp=1:t) <= B[i])

# Charging only when plugged in
@constraint(model, charge_when_plugged[i=1:N, j=1:M, t=1:T], c[i,t] <= P[j] * x[i,j,t])

# Unique charging station
@constraint(model, [i=1:N, t=1:T], sum(x[i,j,t] for j=1:M) <= 1)

# Solve the problem
optimize!(model)

# Check if the model was solved
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found.")
    # Access and print the solution
    for i = 1:N
        for t = 1:T
            for j = 1:M
                if value(x[i,j,t]) > 0.5
                    println("Vehicle $i charges at station $j at time $t.")
                end
            end
        end
    end
else
    println("No optimal solution found.")
end


Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2024-04-16
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-1265U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1810 rows, 1440 columns and 8040 nonzeros
Model fingerprint: 0x8846a029
Variable types: 240 continuous, 1200 integer (1200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 8e+01]
Presolve removed 1776 rows and 1440 columns
Presolve time: 0.01s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.01 work units)
Thread count was 1 (of 12 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -

User-ca

In [5]:
using JuMP, Gurobi

# Parameters
T = 24  # Number of time intervals
maxCharge = 100  # Max battery capacity in kWh
chargeRate = 20  # Charging rate in kWh per interval
initialCharge = 20  # Initial battery charge in kWh
requiredCharge = 80  # Required charge level by the end of T to complete the trips

# Decision variables: Binary decision to charge at time t
chargeDecision = zeros(Bool, T)

# DP Table to store minimum total charging time to reach a certain charge level at each time
# Initialize with large numbers representing high "cost"
dpTable = fill(Inf, T+1, maxCharge+1)

# Initial condition: at time 0, the cost to be at initialCharge is 0
dpTable[1, initialCharge+1] = 0

# Dynamic Programming recursion
for t = 1:T
    for chargeLevel = 1:maxCharge+1
        if dpTable[t, chargeLevel] < Inf  # If this state is reachable
            # Option 1: Do not charge
            nextChargeLevel = min(chargeLevel, maxCharge+1)
            dpTable[t+1, nextChargeLevel] = min(dpTable[t+1, nextChargeLevel], dpTable[t, chargeLevel])

            # Option 2: Charge (if possible)
            if chargeLevel + chargeRate <= maxCharge + 1
                nextChargeLevel = min(chargeLevel + chargeRate, maxCharge+1)
                dpTable[t+1, nextChargeLevel] = min(dpTable[t+1, nextChargeLevel], dpTable[t, chargeLevel] + 1)
            end
        end
    end
end

# Backtracking to find the optimal charging decisions
optimalCharge = requiredCharge + 1
for t = T:-1:1
    if dpTable[t, optimalCharge] < dpTable[t+1, optimalCharge]
        # Charging was not beneficial at this step
        chargeDecision[t] = false
    else
        # Charging was beneficial, adjust charge level for backtracking
        chargeDecision[t] = true
        optimalCharge -= chargeRate
        optimalCharge = max(optimalCharge, 1)
    end
end

# Print optimal decisions
println("Optimal charging decisions:")
for t = 1:T
    println("At time $t: ", chargeDecision[t] ? "CHARGE" : "DO NOT CHARGE")
end


Optimal charging decisions:
At time 1: CHARGE
At time 2: CHARGE
At time 3: CHARGE
At time 4: CHARGE
At time 5: CHARGE
At time 6: CHARGE
At time 7: CHARGE
At time 8: CHARGE
At time 9: CHARGE
At time 10: CHARGE
At time 11: CHARGE
At time 12: CHARGE
At time 13: CHARGE
At time 14: CHARGE
At time 15: CHARGE
At time 16: CHARGE
At time 17: CHARGE
At time 18: CHARGE
At time 19: CHARGE
At time 20: CHARGE
At time 21: CHARGE
At time 22: CHARGE
At time 23: CHARGE
At time 24: CHARGE


In [6]:
using JuMP, Gurobi

# Parameters (Simplified Example)
N = 3 # Number of vehicles
M = 2 # Number of charging stations
T = 24 # Time intervals (e.g., hours)
B = [100, 90, 80] # Battery capacities for each vehicle
E = 0.3 # Energy consumption rate (kWh per mile)
P = [20, 25] # Charging rate per hour for each station
D = [10, 20, 15] # Distance for next trip per vehicle (simplified)
S = [1, 1] # Number of slots at each station

# State Space: For simplification, we consider the state as the charge level
# This could be expanded to include location, time until next trip, etc.
max_charge_level = maximum(B)
min_charge_level = 0
charge_step = 10 # Discretization step for charge level

# Value Function: Initializing a 3D array to store the minimum charging time
# Dimension 1: Vehicle Index, Dimension 2: Time Interval, Dimension 3: Charge Level
V = fill(Inf, (N, T+1, (max_charge_level-min_charge_level)÷charge_step + 1))

# Terminal Condition: At the end of the horizon, no more charging time is needed
V[:, T+1, :] .= 0

# DP Recursive Function
function dp_charge(i, t, s)
    if t > T
        return 0
    end
    # Convert charge level to index
    s_idx = (s - min_charge_level) ÷ charge_step + 1
    
    # If already computed, return the value
    if V[i, t, s_idx] != Inf
        return V[i, t, s_idx]
    end
    
    # Iterate over charging stations and charge amounts
    for j in 1:M
        for c in 0:charge_step:min(P[j], B[i]-s)
            new_s = s + c
            if new_s >= E*D[i] # Ensure enough charge for the trip
                # Recurse to the next time interval
                future_cost = dp_charge(i, t+1, new_s)
                # Update the value function if this path is better
                V[i, t, s_idx] = min(V[i, t, s_idx], 1 + future_cost)
            end
        end
    end
    
    return V[i, t, s_idx]
end

# Solve for each vehicle and initial state
for i in 1:N
    initial_charge = 80 # Define or read the initial charge level
    println("Minimum charging time for vehicle $i: ", dp_charge(i, 1, initial_charge))
end


Minimum charging time for vehicle 1: 24.0
Minimum charging time for vehicle 2: 24.0
Minimum charging time for vehicle 3: 24.0
